<a href="https://colab.research.google.com/github/StetskoSergey/algotrade/blob/main/Backtesting_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # библиотека нампи
import pandas as pd # библиотека пандас
import matplotlib.pyplot as plt # библиотека матплотлиб для отрисовки
from IPython.display import clear_output # очистка вывода в ячейке
import datetime # модуль для работы с datetime

In [ ]:
!pip install yfinance -q # установка yfinance
import yfinance as yf # requires yfinance - pip install yfinance
clear_output()

In [ ]:
!pip install backtesting -q
# Библиотеки backtesting 
from backtesting import Backtest, Strategy
clear_output()

# **Backtesting.py**

Бэктестинг — это необходимый компонент в работе трейдера — процесс оценки эффективности торговой системы на основе исторических данных. Бэктестинг дает возможность понять насколько эффективна стратегия во времени, на каких рынках она показывает лучшие результаты, а где он просто теряет деньги.



Backtest trading strategies in Python

[документация](https://kernc.github.io/backtesting.py/)

[примеры](https://kernc.github.io/backtesting.py/doc/backtesting/#example-strategies)

<img src = 'https://kernc.github.io/backtesting.py/logo.png'>